In [1]:
import pandas as pd
import os
import glob
import numpy as np
pd.options.mode.copy_on_write=True

In [ ]:
df_2020 = pd.read_parquet( "../src/data/2020_Accidentalidad_Limpio.parquet", engine="pyarrow" )

df_2021 = pd.read_parquet( "../src/data/2021_Accidentalidad_Limpio.parquet", engine="pyarrow" )

df_2022 = pd.read_parquet( "../src/data/2022_Accidentalidad_Limpio.parquet", engine="pyarrow" )

df_2023 = pd.read_parquet( "../src/data/2023_Accidentalidad_Limpio.parquet", engine="pyarrow" )

df_2024 = pd.read_parquet( "../src/data/2024_Accidentalidad_Limpio.parquet", engine="pyarrow" )



In [3]:
df_2021["origen"] = "2021"
df_2022["origen"] = "2022"
df_2023["origen"] = "2023"
df_2024["origen"] = "2024"


In [4]:
df_accidentes = pd.concat(
    [df_2021, df_2022, df_2023, df_2024],
    ignore_index=True
)


In [5]:
df_accidentes.info()
df_accidentes.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184929 entries, 0 to 184928
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   num_expediente        184929 non-null  object  
 1   fecha                 184929 non-null  object  
 2   hora                  184929 non-null  object  
 3   localizacion          184929 non-null  object  
 4   numero                184929 non-null  object  
 5   cod_distrito          184929 non-null  float64 
 6   distrito              184929 non-null  object  
 7   tipo_accidente        184929 non-null  object  
 8   estado_meteorológico  184929 non-null  object  
 9   tipo_vehiculo         184929 non-null  object  
 10  tipo_persona          184929 non-null  object  
 11  rango_edad            184929 non-null  object  
 12  sexo                  184929 non-null  object  
 13  cod_lesividad         184929 non-null  float64 
 14  lesividad             184929 non-nul

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora,franja_horaria,origen,grado_lesividad
0,2020s019534,2021-01-01,04:30:00,avda. pablo neruda / call. leoneses,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,...,14.0,Sin asistencia sanitaria,444976.0,4470508.0,No,No consta,2021-01-01 04:30:00,Madrugada,2021,NaN
1,2020s019534,2021-01-01,04:30:00,avda. pablo neruda / call. leoneses,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,...,7.0,Leve,444976.0,4470508.0,No,No consta,2021-01-01 04:30:00,Madrugada,2021,NaN
2,2020s019534,2021-01-01,04:30:00,avda. pablo neruda / call. leoneses,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,...,14.0,Sin asistencia sanitaria,444976.0,4470508.0,No,No consta,2021-01-01 04:30:00,Madrugada,2021,NaN
3,2020s019534,2021-01-01,04:30:00,avda. pablo neruda / call. leoneses,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,...,14.0,Sin asistencia sanitaria,444976.0,4470508.0,No,No consta,2021-01-01 04:30:00,Madrugada,2021,NaN
4,2020s019534,2021-01-01,04:30:00,avda. pablo neruda / call. leoneses,57,13.0,PUENTE DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,...,14.0,Sin asistencia sanitaria,444976.0,4470508.0,No,No consta,2021-01-01 04:30:00,Madrugada,2021,NaN


In [19]:
def preparar_accidentes_unicos(df, col_id='num_expediente'):
    """
    Devuelve un DataFrame con accidentes únicos (1 fila = 1 expediente),
    agregando variables relevantes y derivando variables temporales.
    """

    # Asegurar datetime
    df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], errors='coerce')

    # Agregación por expediente
    df_unicos = df.groupby(col_id).agg({
        'fecha_hora': 'min',
        'tipo_accidente': 'first',
        'tipo_vehiculo': lambda x: list(x.unique()),
        'distrito': 'first',
        'estado_meteorológico': 'first',
        'rango_edad': 'first',
        'tipo_persona': 'first',
        'sexo': 'first',
        'positiva_alcohol': 'first'
    }).reset_index()

    # Variables temporales
    df_unicos['año'] = df_unicos['fecha_hora'].dt.year
    df_unicos['mes'] = df_unicos['fecha_hora'].dt.month
    df_unicos['dia_semana'] = df_unicos['fecha_hora'].dt.day_name()
    df_unicos['hora'] = df_unicos['fecha_hora'].dt.hour

    df_unicos['franja'] = pd.cut(
        df_unicos['hora'],
        bins=[0,6,12,18,24],
        labels=['Madrugada','Mañana','Tarde','Noche'],
        right=False
    )

    return df_unicos


In [20]:
import matplotlib.pyplot as plt
import pandas as pd

def curva_mensual_multianual(df, col_datetime="fecha_hora"):
    """
    Genera una única gráfica con la frecuencia mensual de accidentes,
    mostrando una línea por cada año presente en el DataFrame.
    """

    # Asegurar datetime
    df[col_datetime] = pd.to_datetime(df[col_datetime], errors="coerce")

    # Agrupar por año y mes → tabla año × mes
    tabla = (
        df.groupby([df[col_datetime].dt.year, df[col_datetime].dt.month])
          .size()
          .unstack(fill_value=0)
    )

    # Nombres de meses
    meses_es = [
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    ]
    tabla.columns = meses_es[:len(tabla.columns)]

    # Crear figura
    plt.figure(figsize=(14, 6))

    # Dibujar una línea por año
    for año in tabla.index:
        plt.plot(
            tabla.columns,
            tabla.loc[año],
            marker="o",
            linewidth=3,
            label=str(año)
        )

        # Etiquetas encima de cada punto
        for i, v in enumerate(tabla.loc[año]):
            plt.text(i, v + 5, str(v), ha="center", fontsize=8)

    # Estética
    plt.title("Curva mensual de accidentes por año", fontsize=14)
    plt.xlabel("Mes")
    plt.ylabel("Número de accidentes")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend(title="Año")
    plt.tight_layout()
    plt.show()


In [21]:
df_unicos = preparar_accidentes_unicos(df_accidentes)
curva_mensual_multianual(df_unicos)
